# Project: The Bin-Packing and Knapsack Problems

**For: COMP 215 Spring 2024  
By: Stephen Bowman**

The goal with this notebook is to explain the Bin-packing and Knapsack Problem, while also detailing some of the approximation algorithms used to solve them. As these are both NP-Hard, no known polynomial time algorithms exist for efficiently solving either problem (more on that later). There are, however, many approximation algorithms using heuristics and/or sophisticated algorithms that can generate optimal solutions for very large instances of the problem and get very close to polynomial time.

For those who might want a deeper dive into specific topics discussed I've linked the relevant wikipedia articales that I used when I began my own research. They each serve as good starting points to began learning about each topic and contain useful links to research articles, books, and other even more detailed sources.

**Sources:**

https://en.wikipedia.org/wiki/Bin_packing_problem  
https://en.wikipedia.org/wiki/Knapsack_problem  
https://en.wikipedia.org/wiki/Heuristic_(computer_science)  
https://en.wikipedia.org/wiki/Heuristic  
https://en.wikipedia.org/wiki/Greedy_algorithm  

## Bin Packing

Let's start by illustrating the problem. Say we have a number of items with varying weights, these could be the mass of the items or some arbitray value, and a number of bins (any vessel meant to hold the items) with some max weight they can hold. For simplicity we will assume that each cart has the same weight limit. The question we then pose is how do we pack all the items while using the fewest number of bins. 

Now if this sounds easy, at least for a small problem, you not necessesarily wrong. For very small problems (single to double digit number of items) we can generally solve this by hand. But once we get to hundreds, thousands, millions of items we need a method that can be applied for **any** problem that can be represented as a bin packing problem. We certainly want a method that can find an optimal solution while also doing so in polynomial time, then as the problem gets larger is doesn't become computational infeasible to compute. Basically, if we go from a thousand to one-hundred thousand items, we dont want the time complexity to jump exponentialy (for example), and go from taking a couple minutes to a couple months to solve the problem. 

So what do we do?

### Approximation Algorithms

Thankfully mainy a brilliant researcher in Operations Research, the field in which bin-packing and similar problems are studied, have come up with many useful approximation and heuristic based algorithms. A heuristic, in the context of computer science and operations research, refers to a method of problem solving based on previous experience or knowledge on the problem when no other algorithms exist that can be used to solve the problem and find a sufficient, but maybe not optimal, solution to a problem. Heuristic methods will usually sacrifice optimality, completeness, accuracy, or precision for speed. The study of heuristics and their applications is a diverse and broad area of study that could be it's own entire project for this course, so I'll leave it there with a link at the start of the notebook should you be interested. 

Lets make a small test list of items with some name and weight. 

In [1]:
test = [(2, 'item1'), (10, 'item2'), (4, 'item3'), (4, 'item3'), (10, 'item2'),
       (1, 'item4'),(3, 'item5'),(5, 'item6'),(7, 'item7')]

We now define the max weight that each bin can take.

In [2]:
max_weight = 10

For your own understanding, play around with the values and max weight and see how the solution changes. What happens when the weights are real numbers (1.1234234) vs integers? What if there were no duplicate items in the list?

**A note of the types of algorithms: Online vs Offline**

Before the items are packed most methods will order the items by either descending (largest to smallest) or ascending (smallest to largest) weights, for this work book we will order them  in descending order. There are two general methods for solving, an **online heuristic algorithm or an offline one**. It should be noted that the categories themselves represent the way the algorithm accessess and stores the list of items and bins, not the specific algorithm used for deciding what items go where. Both online and offline methods have multiple algorithms for the decision part of the problem.

From wikipedia, online algorithms have "...the items arrive one after another and the (irreversible) decision where to place an item has to be made before knowing the next item or even if there will be another one." Another way to look at it, the algorithm doesn't see all the items before placing them into bins, and once the bin is considered full, or next item given can't fit into the current bin, it closes the bin, and creates a new one. You can probably see why this won't always give an optimal solution, as if an item that could fit into a bin that comes later in the queue it could end up in it's own bin, so we don't get the fewest number of bins, but something closer.

If we want the approximation to be even closer to optimal, we can make use of an offline algorithm. Also from wikipedia,
"in the offline version of bin packing, the algorithm can see all the items before starting to place them into bins."  
Offline algorithms generally (but not always) follow a similar technique of;
1. "Ordering the input list by descending size" 
2. "Run an online algorithm on the ordered list"  


The algorithm we'll make use of comes from the stackoverflow user "rocksportrocker", with some minor modifications. Link below,

https://stackoverflow.com/questions/7392143/python-implementations-of-packing-algorithm

This solution uses the First-Fit-Decreasing algorithm. First-Fit is a online algorithm that "keeps all bins open, in the order in which they were opened. It attempts to place each new item into the **first** bin in which it fits". The decreasing part of the name is also literal as it refers to the order of the items when/at input.

The below solution uses a python object class (Bin) to act as storage of the bins plus the running sum 

In [3]:
""" Partition a list into sublists whose sums don't exceed a maximum 
    using a First Fit Decreasing algorithm.
"""

class Bin(object):
    """ Container for items that keeps a running sum """
    def __init__(self):
        self.items = []
        self.sum = 0

    def append(self, item):
        self.items.append(item)
        self.sum += item[0]

    def __str__(self):
        """ Printable representation """
        return f'Bin(sum={self.sum}, items={str(self.items)})'


def pack(values, maxValue):
    values = sorted(values,reverse=True)
    bins = []

    for item in values:
        # Try to fit item into a bin
        for bin in bins:
            if bin.sum + item[0] <= maxValue:
                bin.append(item)
                break
        else:
            # item didn't fit into any bin, start a new bin
            bin = Bin()
            bin.append(item)
            bins.append(bin)

    return bins


def packAndShow(aList, maxValue):
    """ Pack a list into bins and show the result """
    print(f'List with sum {sum(x[0] for x in aList)}, requires at least {(sum(x[0] for x in aList)+maxValue-1)/maxValue} bins')

    bins = pack(aList, maxValue)

    print('Solution using', len(bins), 'bins:')
    for bin in bins:
        print(bin)

Now, lets use the list of items and test list and see the above algorithm in action.

In [4]:
packAndShow(test, max_weight)

List with sum 46, requires at least 5.5 bins
Solution using 5 bins:
Bin(sum=10, items=[(10, 'item2')])
Bin(sum=10, items=[(10, 'item2')])
Bin(sum=10, items=[(7, 'item7'), (3, 'item5')])
Bin(sum=10, items=[(5, 'item6'), (4, 'item3'), (1, 'item4')])
Bin(sum=6, items=[(4, 'item3'), (2, 'item1')])


Success! This only scrathes the surface, a quick look at the wikipedia article for bin-packing has algorithms like Next Fit, Next-k-fit, Best-Fit, Next-fit-decreasing, Modified first-fit-decreasing, etc. There are related problems like 2D and 3D bin-packing where the volume and/or dimensions are considered, or others like the inverse bin-packing, maximum resource bin-packing, guillotine cutting, etc.

Hopefully this has shown why bin-packing is so difficult but also heavily researched. But what happens when we only have one bin but want to maximise the items in the value of items in the bin, where the items have both a weight and a value. From bin-packing we now move to the Knapsack problem.

## The Knapsack Problem 



From the associated wikipedia article,  
"Given a set of items, each with a weight and a value, determine which items to include in the collection so that the total weight is less than or equal to a given limit and the total value is as large as possible."

This knapsack in the name comes from the version of the problem where an individual has one knapsack and must fill it with the most valuable items. Similar to bin-packing, the knapsack problem is NP-hard, so while there are no known algorithms that give an optimal and fast solution, there exist many algorithms for finding an approximate solution, some degree close to optimal in a reasonable time. 

One algorithm to solve the knapsack algorithm comes from Dr. George Bernard Dantzig, well known in operations research and developing the simplex method for solving linear programming problems, with contributions the fields of industrial engineering, operations research, computer science, economics, and statistics.  

Dr. Dantzig proposed a greedy approximation algorithm to solve the unbounded knapsack problem, where there are an unlimited number of each item being considered. His version sorts the items in decreasing order of value per unit of weight, this reduction using the value divided by the weight of the object makes it resemble the bin-packing problem. "It then proceeds to insert them into the sack, starting with as many copies as possible of the first kind of item until there is no longer space in the sack for more." However, when the supply of items is limited this algorithm can be far from optimal. The fix for the bounded problem, and for bettering the approximation, modifies the algorithm to make two different lists of solutions and use a concept from mixed integer programming called LP-relaxation. As the previous concepts are beyond my current level of knowledge/skill I will be skipping an indepth analysis. 

**Side note: What is a greedy approximation algorithm** 

From wikipedia, "A greedy algorithm is any algorithm that follows the problem-solving heuristic of making the locally optimal choice at each stage. In many problems, a greedy strategy does not produce an optimal solution, but a greedy heuristic can yield locally optimal solutions that approximate a globally optimal solution in a reasonable amount of time."

In essence, a greedy algorithm uses some heuristic to find an optimal solution for a smaller part of the problem, such as the decisions made in which item is added to a bin. By picking the optimal solution at each step, it approximates and/or finds the optimal solution for the entire problem when each step is added together. This does mean the algorithm only considers the choice infront of it, never reconsidering past choices. Methods like dynamic programming consider all choices made from past to current, but that is a topic for a whole other project and notebook.

Instead of attempting a fully modified greedy algorithm for the knapsack problem we will use the modified code from user Gareth Rees to answer the stack overflow question at,  
https://codereview.stackexchange.com/questions/62344/functional-knapsack-problem-in-python

The below code uses the greedy algorithm initially proposed by Dantzig. While the solution could still be optimal, it's not guaranteed.

In [5]:
from collections import namedtuple

Item = namedtuple('Item', 'name weight value'.split())

ITEMS = [
    Item("map", 9, 150),
    Item("compass", 13, 35),
    Item("water", 153, 200),
    Item("sandwich", 50, 160),
    Item("glucose", 15, 60),
    Item("tin", 68, 45),
    Item("banana", 27, 60),
    Item("apple", 39, 40),
    Item("cheese", 23, 30),
    Item("beer", 52, 10),
    Item("suntan cream", 11, 70),
    Item("camera", 32, 30),
    Item("t-shirt", 24, 15),
    Item("trousers", 48, 10),
    Item("umbrella", 73, 40),
    Item("waterproof trousers", 42, 70),
    Item("waterproof overclothes", 43, 75),
    Item("note-case", 22, 80),
    Item("sunglasses", 7, 20),
    Item("towel", 18, 12),
    Item("socks", 4, 50),
    Item("book", 30, 10),
    ]

def efficiency(item):
    """Return efficiency of item (its value per unit weight)."""
    return float(item.value) / float(item.weight)

def packing(items=ITEMS, max_weight=400):
    """Return a list of items with the maximum value, subject to the
    constraint that their combined weight must not exceed max_weight.

    """    
    def pack(item):
        # Attempt to pack item; return True if successful.
        if item.weight <= pack.max_weight:
            pack.max_weight -= item.weight
            return True
        else:
            return False

    pack.max_weight = max_weight
    return list(filter(pack, sorted(items, key=efficiency, reverse=True)))

Lets run the above algorithm using the preset max weight (400), and testing with a weight one-fourth the size (100). We'll also make use of the example list of items given on the stackoverflow post by Rees.

In [6]:
packing(), packing(max_weight = 100)

([Item(name='map', weight=9, value=150),
  Item(name='socks', weight=4, value=50),
  Item(name='suntan cream', weight=11, value=70),
  Item(name='glucose', weight=15, value=60),
  Item(name='note-case', weight=22, value=80),
  Item(name='sandwich', weight=50, value=160),
  Item(name='sunglasses', weight=7, value=20),
  Item(name='compass', weight=13, value=35),
  Item(name='banana', weight=27, value=60),
  Item(name='waterproof overclothes', weight=43, value=75),
  Item(name='waterproof trousers', weight=42, value=70),
  Item(name='water', weight=153, value=200)],
 [Item(name='map', weight=9, value=150),
  Item(name='socks', weight=4, value=50),
  Item(name='suntan cream', weight=11, value=70),
  Item(name='glucose', weight=15, value=60),
  Item(name='note-case', weight=22, value=80),
  Item(name='sunglasses', weight=7, value=20),
  Item(name='compass', weight=13, value=35),
  Item(name='towel', weight=18, value=12)])

The above algorithm does gives us approximate solutons, and while maybe not optimal, are very close. While there is an algorithm using dynamic programming that give a pseudopolynomial time solution, similar to mized integer, is a bit beyond my level of understanding and could be it's own project notebook.

I hope this shows why the Knapsack problem is both so incredibly interesting but also complicated. Looking to ones own life you might find situations or moments where the knapsack problem reared it's head, but it was small enough (at least hopefully) that through trial and error you found your solution. But, as is the case in much of science and mathematics, the questions of what if and how big can we go, come and open a door to some truly fascinating and complex problems.

## Run Your Own Test

Below is the variables for each problem left blank (with a starter item) for you to test how the algorithms perform depending on the inputs. Enjoy.

In [ ]:
test = [(2, 'item1'), ]

bin_max_weight = 

packAndShow(test, bin_max_weight)

In [ ]:
Item = namedtuple('Item', 'name weight value'.split())

ITEMS_test = [
    Item("map", 9, 150),
  
    ]

knapsack_max_weight = 

packing(items=ITEMS_test, max_weight=knapsack_max_weight)